In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_excel("D:\\Universities Lectures\\NED Lectures\\Machine Learning Classes\\RAW.xlsx")
df.head()

,Unnamed: 0,libelle,prix,quartier,details,imgUrl,description,avantages,lon,lat
0,0,Duplex moderne avec terrasse au triangle d'or,4 200 000 DH,Maârif à Casablanca,213 m² - 9 Pièces - 3 Chambres - 4 Salles de b...,paxaenzoui.png,Alter Foncier vous propose à la vente ce magni...,Orientation: Sud - Terrasse - Garage - Ascense...,-7.636185,33.588910
1,1,Splendide villa à vendre à Anfa. Piscine et do...,46 000 000 DH,Ain Diab à Casablanca,3500 m² - 5 Pièces - 6 Chambres - 2 Salles de ...,siwmbyihrw.png,A VENDRE UNE VILLA A ANFA A RETAPER. VUE MER D...,Jardin - Terrasse - Garage - Vue sur mer - Pis...,-7.612707,33.583335
2,2,"Appartement de 67m² en vente, Ahl Loghlam Soci...",250 000 DH,Ahl Loghlam (Hay Assalam) à Casablanca,67 m² - 4 Pièces - 3 Chambres - 2 Salles de ba...,xfkdzbtnmi.png,Le projet résidentiel Shems Almadina Tit Melli...,Jardin - Terrasse - Ascenseur - Vue sur mer - ...,-7.472037,33.584161
3,3,Appartement à la location sur Bourgogne,1 000 000 DH,Bourgogne Ouest à Casablanca,130 m² - 3 Pièces - 2 Chambres - 2 Salles de b...,jdqjhsnxnm.png,A VENDRE UN APPARTEMENT À BOURGOGNE AVEC TERRA...,Orientation: Est - Terrasse - Garage - Ascense...,-7.675838,33.582734
4,4,"Belle villa 700 m², Polo",8 500 000 DH,Polo à Casablanca,700 m² - 15 Pièces - 7 Chambres - 6 Salles de ...,fzwtlfnrgy.png,Alter Foncier vous propose à la vente cette ma...,Orientation: Sud - Jardin - Terrasse - Garage ...,-7.620113,33.560735


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7495 entries, 0 to 7494
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   7495 non-null   int64  
 1   libelle      7495 non-null   object 
 2   prix         7495 non-null   object 
 3   quartier     7495 non-null   object 
 4   details      7493 non-null   object 
 5   imgUrl       7495 non-null   object 
 6   description  7495 non-null   object 
 7   avantages    6867 non-null   object 
 8   lon          6949 non-null   float64
 9   lat          6949 non-null   float64
dtypes: float64(2), int64(1), object(7)
memory usage: 585.7+ KB


In [4]:
# Drop the 'Unnamed: 0' column
df = df.drop('Unnamed: 0', axis=1)
df = df.drop('imgUrl', axis=1)

In [5]:
# Convert 'prix' column to float
df['prix'] = df['prix'].str.extract('(\d+\.?\d*)').astype(float)

In [6]:
# Convert 'lon' and 'lat' columns to float
df['lon'] = df['lon'].astype(float)
df['lat'] = df['lat'].astype(float)

In [7]:
# # Convert 'quartier' column to lowercase and drop any leading/trailing whitespaces
df['quartier'] = df['quartier'].str.lower().str.strip()


In [8]:
# Check the data types of the columns
print(df.dtypes)

# Filter for string columns
string_columns = df.select_dtypes(include='object').columns
print(string_columns)

libelle         object
prix           float64
quartier        object
details         object
description     object
avantages       object
lon            float64
lat            float64
dtype: object
Index(['libelle', 'quartier', 'details', 'description', 'avantages'], dtype='object')


In [9]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for col in ['libelle', 'description','details','avantages','quartier']:
    df[col] = le.fit_transform(df[col])

In [10]:
df.isna().sum(axis = 0)

libelle          0
prix             0
quartier         0
details          0
description      0
avantages        0
lon            546
lat            546
dtype: int64

In [11]:
from sklearn.impute import KNNImputer

In [12]:
# Impute missing values in lon and lat columns
df['lon'] = df['lon'].fillna(df['lon'].mean())
df['lat'] = df['lat'].fillna(df['lat'].mean())

# Replace original dataframe with imputed values
df = df.fillna(df.mean())


In [13]:
import numpy as np

df['details'].fillna(value='unknown', inplace=True)

df.isna().sum(axis = 0)

libelle        0
prix           0
quartier       0
details        0
description    0
avantages      0
lon            0
lat            0
dtype: int64

In [14]:
# Scale numeric variables using StandardScaler
scaler = StandardScaler()
df[['prix', 'lon', 'lat']] = scaler.fit_transform(df[['prix', 'lon', 'lat']])

In [15]:
df.head()

,libelle,prix,quartier,details,description,avantages,lon,lat
0,3114,-0.532341,144,2713,792,2850,-0.033824,0.057280
1,4039,-0.391750,4,3207,109,1760,-0.002932,0.050379
2,1051,0.291125,0,4553,3503,1058,0.182157,0.051401
3,1841,-0.542384,28,1246,106,1989,-0.086000,0.049635
4,2895,-0.518952,156,4709,805,2703,-0.012677,0.022405


In [16]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

In [17]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop(['prix'], axis=1), df['prix'], test_size=0.2, random_state=42)

In [18]:
# Define the Gradient Boosting Regression model
gbr = GradientBoostingRegressor()

In [19]:
# Define the hyperparameter grid
param_grid = {
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 500],
    'max_depth': [3, 5],
    'min_samples_split': [2, 4]
}

In [20]:
# Perform a grid search to find the best hyperparameters
grid_search = GridSearchCV(gbr, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(),
             param_grid={'learning_rate': [0.01, 0.1], 'max_depth': [3, 5],
                         'min_samples_split': [2, 4],
                         'n_estimators': [100, 500]})

In [21]:
# Use the best model to make predictions on the testing set
y_pred = grid_search.predict(X_test)
print(y_pred)

[-0.61620232 -0.44663033  0.63823835 ... -0.54264521 -0.26256102
 -0.53715822]


In [22]:
# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)

print(f"Mean squared error: {mse}")
print(f"Best hyperparameters: {grid_search.best_params_}")

Mean squared error: 0.39868529889412596
Best hyperparameters: {'learning_rate': 0.1, 'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 500}


In [23]:
# Initialize and fit the model on training data
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5)
model.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=5)

In [24]:
# Calculate R-squared score on training data
train_score = model.score(X_train, y_train)
print("Training R-squared score:", train_score)

Training R-squared score: 0.7458999856318675


In [25]:
# Calculate R-squared score on testing data
test_score = model.score(X_test, y_test)
print("Testing R-squared score:", test_score)

Testing R-squared score: 0.5601776437492865
